# Super-G Sprite Testing

V. Hunter Adams

#### Import Necessary Libraries

In [23]:
import serial
import numpy
from matplotlib import pyplot as plt
from IPython.display import clear_output
from IPython.display import Latex
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import HTML 
from IPython.display import HTML
from ipywidgets import *
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
import pandas as pd
import plotly.tools as tls
import urllib2
tls.set_credentials_file(username='v.h.adams', api_key='werEw55EhyYuY92HZtTt')

In [10]:
ser = serial.Serial('/dev/cu.usbserial-DN01AWD9', 9600)
filename="./Prior_Data/sprite_20.txt"

In [9]:
datafile=open(filename, 'a')
count = 0;
while count < 200:
    datafile=open(filename, 'a')
    data = ser.readline().rstrip()
    datafile.write(data)
    datafile.write('\n')
    print data
    count += 1
    datafile.close()
datafile.close()
ser.close()
clear_output()

In [67]:
def createHisto(data, label):
    return go.Histogram(
                x=data,
                histnorm='count',
                name=label,
                autobinx=True,
                opacity=0.75
            )

In [108]:
total=40
data = []
count = 0
means = []
stds = []
zmeandata = []

layout = go.Layout(
    barmode='stack',
    xaxis=dict(
        title='Measured Value'
    ),
    yaxis=dict(
        title='Count'
    ),
    bargap=0.1,
    bargroupgap=0.1
)

In [129]:
def gatherData(index):
    global data
    global count
    global means
    global stds
    global zmeandata
    count = 0
    data, means, stds, zmeandata = [], [], [], []
    for filename in os.listdir('./Prior_Data'):
        x = []
        count += 1
        with open('./Prior_Data/'+filename, "rt") as f:
            for line in f:
                try:
                    if len(line.split(',')) == 12:
                        x.extend([int(line.split(',')[index])])
                    else:
                        continue
                except:
                    continue
        data.extend([createHisto(x, 'Sprite '+str(count))])
        means.extend([numpy.mean(x)])
        stds.extend([numpy.std(x)])
        temp = list(numpy.array(x) - numpy.mean(x))
        zmeandata.extend([createHisto(temp, 'Sprite '+str(count))])
#         print 'Sprite Number '+str(count)
#         print '------------------------------'
#         print 'Mean: '+str(numpy.mean(x))
#         print 'Std: '+str(numpy.std(x)) + '\n\n'
        
    print 'Mean of means: '+str(numpy.mean(means))
    print 'Std of means: '+str(numpy.std(means))
    print 'Mean of Std\'s: '+str(numpy.mean(stds))
    print 'Std of std\'s: '+str(numpy.std(stds))
    print 'Number functioning: '+str(count)
    print 'Number dead: '+str(total-count)

## Gyroscope

In [130]:
gatherData(1)

Mean of means: 562.155195422
Std of means: 667.921070266
Mean of Std's: 41.0199343581
Std of std's: 19.9695590355
Number functioning: 20
Number dead: 20


In [131]:
layout.title='Measured Gyroscope x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gx')

In [132]:
layout.title = 'Calibrated Gyroscope x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gx_zm')

In [133]:
gatherData(2)

Mean of means: -431.654207413
Std of means: 926.693726173
Mean of Std's: 44.9069638104
Std of std's: 14.5785703544
Number functioning: 20
Number dead: 20


In [134]:
layout.title='Measured Gyroscope y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gy')

In [135]:
layout.title = 'Calibrated Gyroscope y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gy_zm')

In [136]:
gatherData(3)

Mean of means: 149.448218212
Std of means: 625.851307682
Mean of Std's: 34.8155305127
Std of std's: 10.7259785255
Number functioning: 20
Number dead: 20


In [137]:
layout.title='Measured Gyroscope z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_gz')

In [138]:
layout.title = 'Calibrated Gyroscope z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_gz_zm')

## Accelerometer

In [139]:
gatherData(4)

Mean of means: 407.441309642
Std of means: 19.6098813507
Mean of Std's: 1.06742338845
Std of std's: 0.120286594894
Number functioning: 20
Number dead: 20


In [140]:
layout.title='Measured Accelerometer x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_ax')

In [141]:
layout.title = 'Calibrated Accelerometer x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ax_zm')

In [142]:
gatherData(5)

Mean of means: -15.3603056063
Std of means: 13.5646532975
Mean of Std's: 0.634385783069
Std of std's: 0.0591360635545
Number functioning: 20
Number dead: 20


In [143]:
layout.title='Measured Accelerometer y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_ay')

In [144]:
layout.title = 'Calibrated Accelerometer y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ay_zm')

In [145]:
gatherData(6)

Mean of means: -927.717516573
Std of means: 17.5250809247
Mean of Std's: 2.17795342054
Std of std's: 0.378372705358
Number functioning: 20
Number dead: 20


In [146]:
layout.title='Measured Accelerometer z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_az')

In [147]:
layout.title = 'Calibrated Accelerometer z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_az_zm')

## Magnetometer

In [148]:
gatherData(7)

Mean of means: 710.794392391
Std of means: 119.194462054
Mean of Std's: 10.6573828387
Std of std's: 0.693211385921
Number functioning: 20
Number dead: 20


In [149]:
layout.title='Measured Magnetometer x-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_mx')

In [150]:
layout.title = 'Calibrated Accelerometer x-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ax_zm')

In [151]:
gatherData(8)

Mean of means: 42.928382912
Std of means: 128.929381413
Mean of Std's: 7.15595651799
Std of std's: 0.54419568913
Number functioning: 20
Number dead: 20


In [152]:
layout.title='Measured Magnetometer y-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_my')

In [153]:
layout.title = 'Calibrated Accelerometer y-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_ay_zm')

In [154]:
gatherData(9)

Mean of means: 184.594469418
Std of means: 63.1629451867
Mean of Std's: 6.15782769746
Std of std's: 0.39378798435
Number functioning: 20
Number dead: 20


In [155]:
layout.title='Measured Magnetometer z-Values'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_mz')

In [156]:
layout.title = 'Calibrated Accelerometer z-Values'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_az_zm')

## Phototransistors

In [157]:
gatherData(10)

Mean of means: 148.514424725
Std of means: 18.2053085801
Mean of Std's: 4.2231974634
Std of std's: 1.22570075664
Number functioning: 20
Number dead: 20


In [160]:
layout.title='Measured Rear Light Sensor'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_lr')

In [161]:
layout.title = 'Calibrated Rear Light Sensor'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_lr_zm')

In [162]:
gatherData(11)

Mean of means: 2.5373806152
Std of means: 0.906281228231
Mean of Std's: 2.6322225708
Std of std's: 0.347509174878
Number functioning: 20
Number dead: 20


In [163]:
layout.title='Measured Front Light Sensor'
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='sprite_tests_lf')

In [164]:
layout.title = 'Calibrated Front Light Sensor'
fig2 = go.Figure(data=zmeandata, layout=layout)
py.iplot(fig2, filename='sprite_tests_lf_zm')